In [1]:
!pip install transformers

     ---------------------------------------- 7.0/7.0 MB 91.6 kB/s eta 0:00:00
     ------------------------------------- 200.1/200.1 kB 75.5 kB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 86.7 kB/s eta 0:00:00


In [2]:
import torch
from transformers import DebertaTokenizer, DebertaForSequenceClassification
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm


C:\Users\hp\.conda\envs\tf-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base').to(device)


In [ ]:
train_df = pd.read_csv(r"C:\Users\hp\Downloads\train_nltk_all.csv", usecols=['TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'PRODUCT_TYPE_ID', 'PRODUCT_LENGTH'])
train_df = train_df.dropna()
train_df = train_df.sample(n=100000, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(train_df[['TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'PRODUCT_TYPE_ID']], train_df['PRODUCT_LENGTH'], test_size=0.2, random_state=42)


In [6]:
model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=1)

In [7]:
X_train_encoded = tokenizer(X_train.to_numpy().tolist(), padding=True, truncation=True, return_tensors='pt').to(device)
y_train_tensor = torch.tensor(y_train.to_numpy()).to(device)
X_val_encoded = tokenizer(X_val.to_numpy().tolist(), padding=True, truncation=True, return_tensors='pt').to(device)
y_val_tensor = torch.tensor(y_val.to_numpy()).to(device)


In [8]:
y_train_tensor = torch.tensor(y_train.to_numpy())
y_val_tensor = torch.tensor(y_val.to_numpy())


In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()


,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,none,none,none,1650,2125.980000
1,2673191,none,none,none,2755,393.700000
2,2765088,none,none,none,7537,748.031495
3,1594019,none,none,none,2996,787.401574
4,283658,none,none,none,6112,598.424000


In [11]:
train_dataset = TensorDataset(X_train_encoded['input_ids'], X_train_encoded['attention_mask'], y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32)


In [17]:
train_sample = train_df.sample(frac = 0.1, random_state = 42)
test_sample = test_df.sample(frac = 0.1, random_state = 42)

In [18]:
train_sample = pd.get_dummies(train_sample, columns=['PRODUCT_TYPE_ID'])
test_sample = pd.get_dummies(test_sample, columns=['PRODUCT_TYPE_ID'])

In [19]:
X_train, X_val, y_train, y_val = train_test_split(train_sample.drop(['PRODUCT_ID', 'PRODUCT_LENGTH'], axis=1), train_sample['PRODUCT_LENGTH'], test_size=0.2, random_state=42)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Use TF-IDF to convert the text data into numerical vectors
tfidf = TfidfVectorizer(max_features=5000)
tfidf.fit(X_train['TITLE'] + ' ' + X_train['DESCRIPTION'] + ' ' + X_train['BULLET_POINTS'])

TfidfVectorizer(max_features=5000)

In [21]:
X_train_tfidf = tfidf.transform(X_train['TITLE'] + ' ' + X_train['DESCRIPTION'] + ' ' + X_train['BULLET_POINTS'])
X_val_tfidf = tfidf.transform(X_val['TITLE'] + ' ' + X_val['DESCRIPTION'] + ' ' + X_val['BULLET_POINTS'])
X_test_tfidf = tfidf.transform(test_df['TITLE'] + ' ' + test_df['DESCRIPTION'] + ' ' + test_df['BULLET_POINTS'])

In [22]:
X_train = pd.concat([X_train.drop(['TITLE', 'DESCRIPTION', 'BULLET_POINTS'], axis=1), pd.DataFrame(X_train_tfidf.toarray())], axis=1)
X_val = pd.concat([X_val.drop(['TITLE', 'DESCRIPTION', 'BULLET_POINTS'], axis=1), pd.DataFrame(X_val_tfidf.toarray())], axis=1)
X_test = pd.concat([test_df.drop(['PRODUCT_ID'], axis=1), pd.DataFrame(X_test_tfidf.toarray())], axis=1)

MemoryError: Unable to allocate 6.70 GiB for an array with shape (179976, 5000) and data type float64

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error

# Train a baseline model such as linear regression or decision tree
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
score = max(0, 100*(1-mean_absolute_percentage_error(y_val, y_pred)))
print